# Data Monitoring with Flask and whylogs


In this example we will deploy a flask app with an ml model and use whylogs to monitor the data flowing throught the application. In this case we are using a pytorch image classification model based on densenet.


The api will receive a json request with the path of the image file, this path can be locally acesseed by the app or could be an for example a `s3` bucket or another storage supported by `smart_open`. 
The response will be also a `json` with the classification of the image based on ImageNet pretrained model, hence it will be one of the ImageNet classes. 


For this example you will need to install:


In [ ]:
!pip install Flask==2.0.1 torchvision==0.10.0 whylogs==0.6.1 requests==2.26.0

We will begin by loading the model and the `whylogs` logger in memory as the application is launched

```python
from torchvision import models

from whylogs import get_or_create_session

os.environ["WHYLABS_API_KEY"] = "<API-KEY>"
os.environ["WHYLABS_API_ENDPOINT"] = "<end point override. not required>"
os.environ["WHYLABS_DEFAULT_ORG_ID"] = "<your-org-id>"
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = "<your-default-dataset-id>"

session = get_or_create_session()
whylog_logger = session.logger(tags={"datasetId": "<override-dataset-id>"}, , 
                        dataset_timestamp=datetime.datetime.now(datetime.timezone.utc), 
                        with_rotation_time="5s")

imagenet_class_index = json.load(open("imagenet_class_index.json"))
model = models.densenet121(pretrained=True)
model.eval()

app = Flask(__name__)
```






## Receiving Request and Monitoring Inputs

We will set the endpoint for our app to be `/predict`, this will the main interactions with our application, hence it is important to monitor all the data that is coming in. In this case the request json will include a path to an image.


```python
@app.route("/predict", methods=["POST"])
def predict():

    if request.method == "POST":
        filepath=request.json.get("file",None) 
        image = fetch_file(filepath)
        
        whylog_logger.log({"filepath":filepath })
        whylog_logger.log_image(image)
```



```python

```

In [54]:
!python my_flask_app.py

WARN: Missing config
 * Serving Flask app 'my_flask_app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
WARN: Missing config
 * Debugger is active!
 * Debugger PIN: 372-694-894
^C


In [52]:
import requests
from torchvision import transforms

resp = requests.post("http://localhost:5000/predict",
                     json={"file": '../data/flower2.jpg'})
resp.json()

{'class_id': 'n02280649', 'class_name': 'cabbage_butterfly'}

In [46]:
import glob 
files=glob.glob("output/*/*/*/*.bin")
sorted(files)
files

['output/my_deployed_model/e5f8620a-8f15-4d41-a0e9-e9845c317377/protobuf/dataset_profile.2021-08-25_13-15-22.bin',
 'output/my_deployed_model/e5f8620a-8f15-4d41-a0e9-e9845c317377/protobuf/dataset_profile.2021-08-25_13-15-52.bin',
 'output/my_deployed_model/e5f8620a-8f15-4d41-a0e9-e9845c317377/protobuf/dataset_profile.2021-08-25_13-16-22.bin',
 'output/my_deployed_model/e5f8620a-8f15-4d41-a0e9-e9845c317377/protobuf/dataset_profile.2021-08-25_13-16-52.bin']

In [47]:
from whylogs import DatasetProfile
profile=DatasetProfile.read_protobuf(files[-1])

In [48]:
from whylogs.viz import profile_viewer

In [49]:
profile_viewer(profiles=[profile])

'/var/folders/pr/f715zv8x17b1v5vwydgv2gq40000gq/T/tmpr3be6_r7.html'